<a href="https://colab.research.google.com/github/janoudi/datasharing/blob/master/Veeva_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import csv
import pandas as pd

Scenario analysis for Veeva EDC. First cell represent fixed values. Initial values reflect the Veeva proposal

In [1]:

patient_block = 300
block_duration_days = 61
block_duration_months = 3
first_tier = 100
second_tier = 200
third_tier = 300
cost_first_tier = 50
cost_second_tier = 40
cost_third_tier = 35
cost_passive = 5
cap_per_block = 10000
cap_per_month = 25000
base_cost_per_month = 2000
days_per_month = 30
build_cost = 2000


Input parameters. The two factors that will drive the overall costs are study duration and total number of patients enrolled. While our perpetual trials do not have either pre-planned, a scenario analysis can offer infomation about costs.

In [24]:
study_expected_duration_month = 24
total_number_patients = 6000

Study overall charactersitcs based on input paramaters

In [25]:
enrolled_per_day = total_number_patients / (study_expected_duration_month * days_per_month)
enrolled_per_month = total_number_patients / study_expected_duration_month
days_to_block_size = patient_block / enrolled_per_day
months_to_block_size = patient_block / enrolled_per_month
days_to_block_passive = days_to_block_size + block_duration_days
months_to_block_passive = months_to_block_size + block_duration_months
total_number_blocks = total_number_patients / patient_block

Defining a function to calculate the cost of an active patient block based on the tier system and the cap. Note to self - base cost should be calcualted here

In [26]:
def active_pateints_total_cost (active_patients_n, first_tier_n = first_tier, second_tier_n = second_tier, third_tier_n = third_tier, first_tier_cost = cost_first_tier, second_tier_cost = cost_second_tier, third_tier_cost = cost_third_tier, base_cost = base_cost_per_month):
  if active_patients_n < first_tier_n:
    cost_for_active_patients = active_patients_n * first_tier_cost
    if cost_for_active_patients + base_cost < cap_per_block:
      return cost_for_active_patients + base_cost
    else:
      return cap_per_block
  elif active_patients_n < second_tier_n:
    cost_for_active_patients = active_patients_n * second_tier_cost
    if cost_for_active_patients + base_cost < cap_per_block:
      return cost_for_active_patients + base_cost
    else:
      return cap_per_block
  elif active_patients_n <= third_tier_n:
    cost_for_active_patients = active_patients_n * third_tier_cost
    if cost_for_active_patients + base_cost < cap_per_block:
      return cost_for_active_patients + base_cost
    else:
      return cap_per_block
  else:
    return "Error: number of active patients is invalid"

Loop to calculate per-month costs

In [27]:
per_month_patients = []
per_month_cost = []
month = []
per_month_info = {}
n = 1
while n <= study_expected_duration_month:
  new_patients = enrolled_per_month
  if len(per_month_patients) > 0:
    old_patients = per_month_patients[(n-2)]
  else:
    old_patients = 0
  all_patients = new_patients + old_patients
  per_month_patients.append(all_patients)
  number_of_completed_blocks = int(all_patients / patient_block) 
  if number_of_completed_blocks <= 1:
    number_of_passive_blocks = int(n / months_to_block_passive)
  else:
    number_of_passive_blocks = int((n - block_duration_months)/ months_to_block_size)  
  number_of_completed_active_blocks = number_of_completed_blocks - number_of_passive_blocks
  number_of_active_patients_not_in_block = all_patients - (number_of_completed_blocks * patient_block)
  if number_of_active_patients_not_in_block > 0:
    cost_for_active_patients_not_in_block = active_pateints_total_cost(number_of_active_patients_not_in_block)
  else:
    cost_for_active_patients_not_in_block = 0
  cost_for_active_blocks = number_of_completed_active_blocks * cap_per_block
  cost_for_passive_patients = number_of_passive_blocks * patient_block * cost_passive
  total_costs_per_month = cost_for_active_patients_not_in_block + cost_for_active_blocks + cost_for_passive_patients
  if total_costs_per_month < cap_per_month:
    monthly_total = total_costs_per_month
    per_month_cost.append(total_costs_per_month)
  else:
    monthly_total = cap_per_month
    per_month_cost.append(cap_per_month)
  month.append(n)
  per_month_info[n] = {
      'number of new patients': new_patients,
      'number of old patients': old_patients,
      'total number of patients': all_patients,
      'number of completed blocks': number_of_completed_blocks,
      'number of passive block': number_of_passive_blocks,
      'number of completed active blocks': number_of_completed_active_blocks,
      'number of active patients not in a completed block' : number_of_active_patients_not_in_block,
      'number of passive patients': number_of_passive_blocks,
      'cost for active patients not in a block': cost_for_active_patients_not_in_block,
      'cost for active blocks': cost_for_active_blocks,
      'cost for passive patients': cost_for_passive_patients,
      'total cost for the month': monthly_total
  }
  n = n + 1





       





In [29]:
print(sum(per_month_cost))

579000.0


In [30]:
print(per_month_info)

{1: {'number of new patients': 250.0, 'number of old patients': 0, 'total number of patients': 250.0, 'number of completed blocks': 0, 'number of passive block': 0, 'number of completed active blocks': 0, 'number of active patients not in a completed block': 250.0, 'number of passive patients': 0, 'cost for active patients not in a block': 10000, 'cost for active blocks': 0, 'cost for passive patients': 0, 'total cost for the month': 10000}, 2: {'number of new patients': 250.0, 'number of old patients': 250.0, 'total number of patients': 500.0, 'number of completed blocks': 1, 'number of passive block': 0, 'number of completed active blocks': 1, 'number of active patients not in a completed block': 200.0, 'number of passive patients': 0, 'cost for active patients not in a block': 9000.0, 'cost for active blocks': 10000, 'cost for passive patients': 0, 'total cost for the month': 19000.0}, 3: {'number of new patients': 250.0, 'number of old patients': 500.0, 'total number of patients': 

In [33]:
df = pd.DataFrame(per_month_info)

In [34]:
df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
number of new patients,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,...,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0
number of old patients,0.0,250.0,500.0,750.0,1000.0,1250.0,1500.0,1750.0,2000.0,2250.0,...,3500.0,3750.0,4000.0,4250.0,4500.0,4750.0,5000.0,5250.0,5500.0,5750.0
total number of patients,250.0,500.0,750.0,1000.0,1250.0,1500.0,1750.0,2000.0,2250.0,2500.0,...,3750.0,4000.0,4250.0,4500.0,4750.0,5000.0,5250.0,5500.0,5750.0,6000.0
number of completed blocks,0.0,1.0,2.0,3.0,4.0,5.0,5.0,6.0,7.0,8.0,...,12.0,13.0,14.0,15.0,15.0,16.0,17.0,18.0,19.0,20.0
number of passive block,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,5.0,...,10.0,10.0,11.0,12.0,13.0,14.0,15.0,15.0,16.0,17.0
number of completed active blocks,0.0,1.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,...,2.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0
number of active patients not in a completed block,250.0,200.0,150.0,100.0,50.0,0.0,250.0,200.0,150.0,100.0,...,150.0,100.0,50.0,0.0,250.0,200.0,150.0,100.0,50.0,0.0
number of passive patients,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,5.0,...,10.0,10.0,11.0,12.0,13.0,14.0,15.0,15.0,16.0,17.0
cost for active patients not in a block,10000.0,9000.0,8000.0,6000.0,4500.0,0.0,10000.0,9000.0,8000.0,6000.0,...,8000.0,6000.0,4500.0,0.0,10000.0,9000.0,8000.0,6000.0,4500.0,0.0
cost for active blocks,0.0,10000.0,20000.0,30000.0,30000.0,30000.0,20000.0,20000.0,20000.0,30000.0,...,20000.0,30000.0,30000.0,30000.0,20000.0,20000.0,20000.0,30000.0,30000.0,30000.0


In [35]:
file_name = 'Veeva_EDC_price_24Months_6000Patients.xlsx'
df.to_excel(file_name)
